In [5]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# This line will add a button to toggle visibility of code blocks, for use with the HTML export version
di.display_html('''<button onclick="jQuery('.input_area').toggle(); jQuery('.prompt').toggle();">Toggle code</button>''', raw=True)

Toggle code

# ARES, Penn Library Course Reserves Issue
> Summary of Issue: the ARES LTI which embeds library references in canvas is not visible for the 'Observer' role. However in batch updates, the 'Observer' role should be mapped to student. 

## References
> https://canvas.instructure.com/doc/api/external_tools.html
> https://canvas.instructure.com/doc/api/file.masquerading.html
> https://canvas.instructure.com/doc/api/file.navigation_tools.html      
> API for accessing and configuring external tools on accounts and courses. "External tools" are IMS LTI links:   
>> http://www.imsglobal.org/developers/LTI/index.cfm    

> **visibility**: 'public', 'members', 'admins':  This specifies what types of users will see the link in the course navigation. "public" means anyone accessing the course, "members" means only users enrolled in the course, and "admins" means only Teachers, TAs, Designers and account admins will see the link. ([source](https://canvas.instructure.com/doc/api/file.navigation_tools.html))


Course:  https://upenn.test.instructure.com/courses/1418936  
What the view should be:

<img src='../images/ares_issue.png' width="650" height="650" >

When one masquerades as a user with the 'Observer' Role in the Canvas side navigation UI 
the LTI is not permitting access so it is not showing up in the sidebar. However, one can still access the tool with the URL:

<img src="../images/ares_observer.png" width="650" height="650">


### Investigation into permissions
It appears that there is no authentication issue for 'Observers' with ARES since they can access with the URL. The question then becomes why aren't 'Observer's given permission to see the navigation tab to the tool in Canvas. The UI of the navigation setting is rather limited and only defines visibility in terms of student and instructor roles. 

#### Let's see what info about these permissions I can figure out with the API


In [3]:
# just some important imports ,, run this cell when u start 
import json
import re
import requests
from pprintpp import pprint as pp
import ipywidgets as widgets
from IPython.core.display import display, HTML
import time

with open('live_envir.json') as json_f:
    data = json.load(json_f)
    key = data['api_key']
    print("...loaded my key and did some package imports...")
    
domain = 'https://upenn.test.instructure.com'
headers = {
    'Authorization': 'Bearer %s' % (key) ,
}

...loaded my key and did some package imports...


#### Let's print out a list of the current tabs for the course

In [9]:
#https://canvas.instructure.com/doc/api/tabs.html
course_number = '1418936'
external_tool_id = '62901'
url = '%s/api/v1/courses/%s/tabs' % (domain,course_number)

response = requests.get(url, headers=headers).json()
pp(response)

[
    {
        'full_url': 'https://upenn.test.instructure.com/courses/1418936',
        'html_url': '/courses/1418936',
        'id': 'home',
        'label': 'Home',
        'position': 1,
        'type': 'internal',
        'visibility': 'public',
    },
    {
        'full_url': 'https://upenn.test.instructure.com/courses/1418936/announcements',
        'html_url': '/courses/1418936/announcements',
        'id': 'announcements',
        'label': 'Announcements',
        'position': 2,
        'type': 'internal',
        'visibility': 'public',
    },
    {
        'full_url': 'https://upenn.test.instructure.com/courses/1418936/assignments',
        'html_url': '/courses/1418936/assignments',
        'id': 'assignments',
        'label': 'Assignments',
        'position': 3,
        'type': 'internal',
        'visibility': 'public',
    },
    {
        'full_url': 'https://upenn.test.instructure.com/courses/1418936/files',
        'html_url': '/courses/1418936/files',
        'id


#### Unpacking the result

For the 'Penn Library Course Reserves' navigation, the meta data is:

```
{
    'full_url': 'https://upenn.test.instructure.com/courses/1418936/external_tools/62901',
    'html_url': '/courses/1418936/external_tools/62901',
    'id': 'context_external_tool_62901',
    'label': ' Penn Library Course Reserves',
    'position': 8,
    'type': 'external',
    'url': 'https://upenn.test.instructure.com/api/v1/courses/1418936/external_tools/sessionless_launch?id=62901&launch_type=course_navigation',
    'visibility': 'members',
}
```

**Note** the 'visibility' is set to members whereas the other tabs are set to public. 



In [5]:
# Let's try to update a tab for a course ... 
#this probably wont work because 'visibility' isnt a documented parameter

tab_id = 'context_external_tool_62901'
course_number = '1418936'
external_tool_id = '62901'
url = '%s/api/v1/courses/%s/tabs/%s' % (domain,course_number,tab_id)

files= {
    'visibility':(None,'public')
}

response = requests.put(url, headers=headers,files=files).json()
pp(response)

# blerg this doesn't work ..

{
    'full_url': 'https://upenn.test.instructure.com/courses/1418936/external_tools/62901',
    'html_url': '/courses/1418936/external_tools/62901',
    'id': 'context_external_tool_62901',
    'label': ' Penn Library Course Reserves',
    'position': 8,
    'type': 'external',
    'url': 'https://upenn.test.instructure.com/api/v1/courses/1418936/external_tools/sessionless_launch?id=62901&launch_type=course_navigation',
    'visibility': 'members',
}


In [6]:
# Let's look at the info associated i with the tool

course_number = '1418936'
external_tool_id = '62901'
url = '%s/api/v1/courses/%s/external_tools' % (domain,course_number)
files = {
    'include_parents': (None,'true'),
}
response = requests.get(url, headers=headers,files=files).json()
tool = [ x for x in response if x['id']==62901 ]
tool[0]['consumer_key'] = 'confidential'
pp(tool)


[
    {
        'account_navigation': None,
        'assignment_edit': None,
        'assignment_menu': None,
        'assignment_selection': None,
        'assignment_view': None,
        'collaboration': None,
        'consumer_key': 'confidential',
        'course_assignments_menu': None,
        'course_home_sub_navigation': None,
        'course_navigation': {
            'default': 'disabled',
            'label': ' Penn Library Course Reserves',
            'selection_height': 400,
            'selection_width': 800,
            'text': ' Penn Library Course Reserves',
            'visibility': 'members',
        },
        'course_settings_sub_navigation': None,
        'created_at': '2014-08-05T14:26:40Z',
        'custom_fields': {},
        'description': 'View reserved content for this course',
        'discussion_topic_menu': None,
        'domain': None,
        'editor_button': None,
        'file_menu': None,
        'global_navigation': None,
        'homework_submissi

In [7]:
# Let's try to edit the 'visibility' here at the course_id
course_number = '1418936'
external_tool_id = '62901'
url = '%s/api/v1/courses/%s/external_tools/%s' % (domain,course_number,external_tool_id)
files = {
    'user_navigation[visibility]': (None,'public')
}
response = requests.get(url, headers=headers).json()
pp(response)
# we can't even see the meta data for this LTI!
#   this error occurs because the LTI is not installed at the course level...? 

{
    'error_report_id': 411150948,
    'errors': [{'message': 'The specified resource does not exist.'}],
}


In [8]:
# Let's try to edit the 'visibility' at the account level
external_tool_id = '62901'
url = '%s/api/v1/accounts/self/external_tools/%s' % (domain,external_tool_id)
files = {
    'course_navigation[visibility]': (None,'public')
}
response = requests.put(url, headers=headers,files=files).json()
response['consumer_key'] = 'confidential'
pp(response)

#this has fixed the visibility issue for 'Observers' 
#but it should be noted that instead of having the course_navigation[visibility] change ... it deleted ...

{
    'account_navigation': None,
    'assignment_edit': None,
    'assignment_menu': None,
    'assignment_selection': None,
    'assignment_view': None,
    'collaboration': None,
    'consumer_key': 'confidential',
    'course_assignments_menu': None,
    'course_home_sub_navigation': None,
    'course_navigation': {
        'label': 'Penn Library Course Reserves',
        'selection_height': 400,
        'selection_width': 800,
    },
    'course_settings_sub_navigation': None,
    'created_at': '2014-08-05T14:26:40Z',
    'custom_fields': {},
    'description': 'View reserved content for this course',
    'discussion_topic_menu': None,
    'domain': None,
    'editor_button': None,
    'file_menu': None,
    'global_navigation': None,
    'homework_submission': None,
    'id': 62901,
    'link_selection': None,
    'migration_selection': None,
    'module_menu': None,
    'name': 'Penn Library Course Reserves',
    'not_selectable': False,
    'post_grades': None,
    'privacy_lev

## In closing
This has fixed the visibility issue for 'Observers' for each course. 


it should be noted that instead of having the course_navigation \[visibility] change ... the metadata for that attribute was deleted all together. I am unclear why but it may be due to the the initial configuration of the LTI which was rather "hacky".  